In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('quran_id.csv')
df.head()

,id,suraId,verseID,ayahText,indoText,readText
0,0,1,1,بِسْمِ اللّٰهِ الرَّحْمٰنِ الرَّحِيْمِ,"Dengan nama Allah Yang Maha Pengasih, Maha Pen...",bismillāhir-raḥmānir-raḥīm
1,1,1,2,اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ,"Segala puji bagi Allah, Tuhan seluruh alam,",al-ḥamdu lillāhi rabbil-'ālamīn
2,2,1,3,الرَّحْمٰنِ الرَّحِيْمِۙ,"Yang Maha Pengasih, Maha Penyayang,",ar-raḥmānir-raḥīm
3,3,1,4,مٰلِكِ يَوْمِ الدِّيْنِۗ,Pemilik hari pembalasan.,māliki yaumid-dīn
4,4,1,5,اِيَّاكَ نَعْبُدُ وَاِيَّاكَ نَسْتَعِيْنُۗ,Hanya kepada Engkaulah kami menyembah dan hany...,iyyāka na'budu wa iyyāka nasta'īn


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   suraId    100 non-null    int64 
 2   verseID   100 non-null    int64 
 3   ayahText  100 non-null    object
 4   indoText  100 non-null    object
 5   readText  100 non-null    object
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


In [31]:
df.describe()

,id,suraId,verseID
count,100.000000,100.000000,100.000000
mean,49.500000,1.930000,43.990000
std,29.011492,0.256432,28.264085
min,0.000000,1.000000,1.000000
25%,24.750000,2.000000,18.750000
50%,49.500000,2.000000,43.500000
75%,74.250000,2.000000,68.250000
max,99.000000,2.000000,93.000000


In [32]:
# juz30 = df[df['suraId']>=78]
# juz30

In [33]:
from transformers import BertTokenizer, BertModel
import torch

# Memuat pre-trained model BERT dan tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Fungsi untuk menghasilkan embedding BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    # Mengambil rata-rata dari seluruh token untuk mendapatkan vektor 2D
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [34]:
import pandas as pd

# Memuat dataset Al-Qur'an dari file CSV
dataset = pd.read_csv('quran_id.csv')

# Menghasilkan embedding untuk setiap ayat
dataset['embedding'] = dataset['indoText'].apply(lambda x: get_bert_embedding(x))


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

# Fungsi untuk melakukan pencarian
def search_ayah(query, dataset, top_n=5):
    query_embedding = get_bert_embedding(query).reshape(1, -1)
    similarities = cosine_similarity(query_embedding, list(dataset['embedding']))
    
    # Mengambil ayat dengan kesamaan tertinggi
    top_indices = similarities.argsort()[0][-top_n:][::-1]
    return dataset.iloc[top_indices][['suraId', 'verseID', 'ayahText', 'indoText']]

# Contoh pencarian
query = "kasih sayang"
results = search_ayah(query, dataset)
print(results)


    suraId  verseID                                           ayahText  \
3        1        4                           مٰلِكِ يَوْمِ الدِّيْنِۗ   
1        1        2            اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ   
24       2       18     صُمٌّ ۢ بُكْمٌ عُمْيٌ فَهُمْ لَا يَرْجِعُوْنَۙ   
8        2        2  ذٰلِكَ الْكِتٰبُ لَا رَيْبَ ۛ فِيْهِ ۛ هُدًى ل...   
18       2       12  اَلَآ اِنَّهُمْ هُمُ الْمُفْسِدُوْنَ وَلٰكِنْ...   

                                             indoText  
3                            Pemilik hari pembalasan.  
1         Segala puji bagi Allah, Tuhan seluruh alam,  
24  Mereka tuli, bisu dan buta, sehingga mereka ti...  
8   Kitab (Al-Qur'an) ini tidak ada keraguan padan...  
18  Ingatlah, sesungguhnya merekalah yang berbuat ...  


In [39]:
query = "iman"
results = search_ayah(query, dataset)
print(results)

    suraId  verseID                                           ayahText  \
7        2        1                                           الۤمّۤ ۚ   
3        1        4                           مٰلِكِ يَوْمِ الدِّيْنِۗ   
58       2       52  ثُمَّ عَفَوْنَا عَنْكُمْ مِّنْۢ بَعْدِ ذٰلِكَ ...   
1        1        2            اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ   
8        2        2  ذٰلِكَ الْكِتٰبُ لَا رَيْبَ ۛ فِيْهِ ۛ هُدًى ل...   

                                             indoText  
7                                       Alif Lam Mim.  
3                            Pemilik hari pembalasan.  
58  Kemudian Kami memaafkan kamu setelah itu, agar...  
1         Segala puji bagi Allah, Tuhan seluruh alam,  
8   Kitab (Al-Qur'an) ini tidak ada keraguan padan...  


In [40]:
query = "tauhid"
results = search_ayah(query, dataset)
print(results)

    suraId  verseID                                           ayahText  \
1        1        2            اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ   
83       2       77  اَوَلَا يَعْلَمُوْنَ اَنَّ اللّٰهَ يَعْلَمُ مَ...   
43       2       37  فَتَلَقّٰٓى اٰدَمُ مِنْ رَّبِّهٖ كَلِمٰتٍ فَتَ...   
24       2       18     صُمٌّ ۢ بُكْمٌ عُمْيٌ فَهُمْ لَا يَرْجِعُوْنَۙ   
58       2       52  ثُمَّ عَفَوْنَا عَنْكُمْ مِّنْۢ بَعْدِ ذٰلِكَ ...   

                                             indoText  
1         Segala puji bagi Allah, Tuhan seluruh alam,  
83  Dan tidakkah mereka tahu bahwa Allah mengetahu...  
43  Kemudian Adam menerima beberapa kalimat dari T...  
24  Mereka tuli, bisu dan buta, sehingga mereka ti...  
58  Kemudian Kami memaafkan kamu setelah itu, agar...  
